In [1]:
import os
import itertools
import random
import pandas as pd
from itertools import combinations
from datetime import datetime

import date_utils as dt_mng
from data_manager import DataManager
import analyze

dm = DataManager()

class Prop:
    def __init__(self, name, team, stat, threshold, odds, bet_type):
        self.name = name
        self.team = team
        self.stat = stat
        self.n = threshold
        self.odds = odds
        self.bet_type = bet_type
        self.probability = self.get_prop_probability()
        self.ev, self.house_prob = self.get_ev_and_implied_prob()
        self.print_out = f"""
            PLAYER: {self.name}
              STAT: {self.stat}
            THRESH: {self.n}
              ODDS: {self.odds}
              TYPE: {self.bet_type}
              PROB: {self.probability}
                EV: {self.ev}
        HOUSE_PROB: {self.house_prob}
            """
        self.entry = {
            "PLAYER": self.name,
              "TEAM": self.team,
              "STAT": self.stat,
            "THRESH": self.n,
              "ODDS": self.odds,
              "TYPE": self.bet_type,
              "PROB": self.probability,
                "EV": self.ev,
        "HOUSE_PROB": self.house_prob
        }
    
    def get_prop_probability(self, last_n_games=25):
        player_id = dm.get_player_id(self.name)
        data = dm.get_and_save_player_data(player_id, self.name).sort_values(by='date', ascending=False).head(last_n_games).copy()
        # print(data.head())
        if self.bet_type == "over":
            return analyze.estimate_probability_poisson_over(data, self.stat, self.n)
        elif self.bet_type == "under":
            return analyze.estimate_probability_poisson_under(data, self.stat, self.n)
        else:
            raise ValueError("Invalid bet type. Use 'over' or 'under'.")
        
    def get_ev_and_implied_prob(self):
        odds = self.american_to_decimal(self.odds)
        house_probability = analyze.estimate_implied_probability(odds)
        ev = analyze.calculate_ev(self.probability, odds, 5)
        return ev, house_probability
    
    @staticmethod
    def american_to_decimal(american_odds):
        """Convert American odds to decimal odds."""
        if american_odds > 0:
            return 1 + (american_odds / 100)
        else:
            return 1 + (100 / abs(american_odds))
    

def extract_raw_data(file_path): # .csv
    # gets input from A1
    # Sample input text (use the content of your file here)
    raw_input = pd.read_csv(file_path)
    list_of_raw_input = list(raw_input.iloc[:, 0])
    return list_of_raw_input


def load_available_props(file_path):
    
    raw_input = extract_raw_data(file_path)
    stat_names = {
             'PointsSGP': "points",
            'AssistsSGP': "assists",
        'Threes MadeSGP': "fg3m",
           'ReboundsSGP': "rebounds",
   'Field Goals MadeSGP': "fgm",
             'StealsSGP': "steals",
             'BlocksSGP': "blocks",
        }
    #debug stat_name_inputs = extract_raw_data("prop_lines/player_prop_categories.csv")
    players = dm.query_players()
    player_names = [player.name for player in players]
    teams = dm.query_teams()
    team_names = [team.nickname for team in teams]
    row_of_interest = 0
    current_category = None
    current_player = None
    current_team = None
    records = []
    for _, item in enumerate(raw_input):
        if item is None: 
            continue
        if item in stat_names:
            current_category = stat_names[item]
        if item != current_team:
            if item in team_names:
                current_team = item
        if item in player_names:
            current_player = item 
            assert current_team
            record = [current_player, current_team, current_category]
            row_of_interest = 6


        if row_of_interest:
            row_of_interest -= 1
            if row_of_interest < 5:
                record.append(item)
                if row_of_interest == 1:
                    records.append(record)
                    record = []
    df = pd.DataFrame.from_records(records, columns=["player_name", "team", "stat", "over_threshold", "over_odds", "under_threshold", "under_odds"])
    
    df['player_name'] = df['player_name'].astype(str)
    df['team'] = df['team'].astype(str)
    df['stat'] = df['stat'].astype(str)
    df['over_threshold'] = df['over_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['under_threshold'] = df['under_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['over_odds'] = df['over_odds'].astype(int)
    df['under_odds'] = df['under_odds'].astype(int)

    return df


def process_all_csv_files_in_directory(directory):
    combined_records = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            print(file_path)
            df = load_available_props(file_path)
            combined_records.append(df)

    # Combine all dataframes into one if desired
    final_df = pd.concat(combined_records, ignore_index=True)
    return final_df


def get_analyzed_props(available_props):
    props = []
    for _, row in available_props.iterrows():
        for bet_type in ["over", "under"]:
            prop = Prop(
                    name=row["player_name"], 
                    team=row["team"],
                    stat=row["stat"], 
                threshold=row[f"{bet_type}_threshold"], 
                    odds=row[f"{bet_type}_odds"], 
                bet_type=bet_type
                )
            props.append(prop)    

    return props


def generate_heterogenous_combinations(df, n):

    # Generate all combinations of n rows
    combinations = list(itertools.combinations(df.index, n))

    # Function to evaluate heterogeneity of a combination
    def evaluate_heterogeneity(comb, df):
        comb_list = list(comb)
        players = df.loc[comb_list, 'PLAYER']
        stats = df.loc[comb_list, 'STAT']
        teams = df.loc[comb_list, 'TEAM']
        # Calculate a simple heterogeneity score (you can define your own logic)
        player_score = len(set(players))
        stat_score = len(set(stats))
        team_score = len(set(teams))
        return player_score + stat_score + team_score

    # Evaluate all combinations and sort them by heterogeneity score
    comb_scores = [(comb, evaluate_heterogeneity(comb, df)) for comb in combinations]
    comb_scores_sorted = sorted(comb_scores, key=lambda x: x[1], reverse=True)

    # Select the most heterogeneous combinations (you can define how many you want)
    top_combinations = comb_scores_sorted # Top 5 combinations for example

    # Display the most heterogeneous combinations
    for comb, score in top_combinations:
        print(f"Combination: {comb}, Score: {score}")
        print(df.loc[list(comb)])
        print()

    # Optional: Convert combinations to DataFrame
    top_comb_dfs = [(df.loc[list(comb)], score) for comb, score in top_combinations]

    return top_comb_dfs


def generate_unique_combinations(df, combo_length):
    all_combinations = list(combinations(df.index, combo_length))
    
    unique_combinations = set()
    valid_combinations = []
    
    for combo in all_combinations:
        players_in_combo = df.loc[list(combo), 'Player']
        if players_in_combo.duplicated().sum() == 0:
            sorted_combo = tuple(sorted(df.loc[list(combo)].apply(lambda row: (row['Player'], row['Category'], row['Bet']), axis=1)))
            if sorted_combo not in unique_combinations:
                unique_combinations.add(sorted_combo)
                valid_combinations.append(combo)
    
    return valid_combinations




def distribute_props_into_parlays(props_df, n_parlays):
    """
    Distributes props into a specified number of parlays based on their probabilities.

    :param props_df: pandas DataFrame with columns 'player', 'team', 'threshold', 'odds', 'bet_type', 'probability'
    :param n_parlays: Number of parlays to distribute the props into
    :return: List of parlays, where each parlay is a list of props
    """
    # Initialize empty parlays
    parlays = [[] for _ in range(n_parlays)]

    # Calculate total probability
    total_probability = props_df["PROB"].sum()
    print(total_probability)

    # Distribute props into parlays based on their probabilities
    for index, row in props_df.iterrows():
        # Determine how many parlays to distribute this prop into
        print(row['PROB'])
        print(row["PROB"] / 1)
        num_parlays_for_prop = int(row["PROB"] / total_probability * n_parlays)
        num_parlays_for_prop = max(1, num_parlays_for_prop)

        # Randomly select parlays for this prop
        selected_parlays = random.sample(parlays, num_parlays_for_prop)
        for parlay in selected_parlays:
            parlay.append(row.to_dict())

    return parlays


def remove_rows_by_value(df, column_name, values_to_remove):
    """
    Removes rows from the DataFrame where the value in the specified column is in the values_to_remove list.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    column_name (str): The name of the column to check for values to remove.
    values_to_remove (list): A list of values to be removed from the DataFrame.

    Returns:
    pandas.DataFrame: The DataFrame with the specified rows removed.
    """
    # Filter the DataFrame to keep rows where the column value is not in the values_to_remove list
    filtered_df = df[~df[column_name].isin(values_to_remove)]
    return filtered_df


def remove_rows_by_shared_id(df, target_column, values_to_check, id_column):
    """
    Removes all rows from the DataFrame that share the same value in the id_column
    if a value in the target_column is found in the values_to_check list.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    target_column (str): The name of the column to check for values.
    values_to_check (list): A list of values to check in the target_column.
    id_column (str): The name of the column containing the IDs to check for shared values.

    Returns:
    pandas.DataFrame: The DataFrame with the specified rows removed.
    """
    # Identify the rows where the target column contains values from the values_to_check list
    rows_to_remove = df[df[target_column].isin(values_to_check)]

    # Get the unique IDs from the id_column for the identified rows
    ids_to_remove = rows_to_remove[id_column].unique()

    # Filter the DataFrame to remove all rows that share the same IDs in the id_column
    filtered_df = df[~df[id_column].isin(ids_to_remove)]
    return filtered_df


def create_study_packet(parlay_study, folder_name):
    players = parlay_study['PLAYER'].unique()
    game_data_players = []
    for player in players:
        player_id = dm.get_player_id(player)
        player_game_data = dm.get_and_save_player_data(player_id)
        game_data_players.append(player_game_data)

    teams = parlay_study['TEAM'].unique()
    game_data_teams = []
    for team in teams:
        team_id = dm.get_team_id(team)
        team_game_data = dm.get_and_save_team_data(team_id)
        game_data_teams.append(team_game_data)
    props = parlay_study.drop_duplicates()[['PROPS', 'THRESH', 'PARLAY_ID', 'PARLAY_EV', 'PARLAY_PROB', 'HOUSE_PARLAY_PROB', 'PROB', 'HOUSE_PROB', 'EV', 'TO_WIN']]
    
    game_data = dict(zip(players, game_data_players))
    for i, team in enumerate(teams):
        game_data[team] = game_data_teams[i]
    dm.save_as_excel_workbook(game_data, folder_name + "/study_packet")
    props.to_csv(folder_name + "/props.csv")


def generate_candidates(n_props, n_parlays, parlay_len, cycles, filter_players, prop_filter):
    available_props = load_available_props()
    analyzed_props = get_analyzed_props(available_props)
    filtered_df = dm.filter_props(analyzed_props, filter_players, n_props)
    parlays = [distribute_props_into_parlays(filtered_df, n_parlays) for n in range(cycles)]
    parlays = [item for sublist in parlays for item in sublist]
    parlays = [parlay for parlay in parlays if len(parlay) == parlay_len]
    parlays = analyze.analyze_parlays(parlays)
    parlays['PROPS'] = parlays['PLAYER'] + "-" + parlays['STAT'] + "-" + parlays['TYPE']
    parlays = remove_rows_by_shared_id(parlays, 'PROPS', prop_filter, 'PARLAY_ID')
    return parlays 


def select_top_parlays(parlays_df, top_n):
    parlays_df = parlays_df.sort_values(by='PARLAY_EV', ascending=False)
    parlay_ids_sorted_by_ev = parlays_df['PARLAY_ID'].unique()
    top_parlay_ids = parlay_ids_sorted_by_ev[:top_n]
    selected_parlays = parlays_df[parlays_df['PARLAY_ID'].isin(top_parlay_ids)]
    return selected_parlays

def generate_parlays(df, min_props, max_props, force_props=None):
    # Initialize list to store parlay combinations
    parlays = []

    # If force_props is None, default to an empty DataFrame with the same columns as df
    force_props = force_props if force_props is not None else pd.DataFrame(columns=df.columns)
    
    # Identify rows in df that match the included props
    required_indices = []
    for _, row in force_props.iterrows():
        # Build a condition to match each specified field
        condition = (df['PLAYER'] == row['name'])
        if pd.notna(row['stat']):
            condition &= (df['STAT'] == row['stat'])
        if pd.notna(row['bet_type']):
            condition &= (df['TYPE'] == row['bet_type'])
        
        # Find indices of rows that match this condition
        print(condition)
        matching_indices = df.index[condition].tolist()
        required_indices.extend(matching_indices)
    
    # Ensure unique required indices
    required_indices = list(set(required_indices))
    
    # Adjust the min and max number of props to account for required_props
    min_combinations = max(0, min_props - len(required_indices))
    max_combinations = max(0, max_props - len(required_indices))

    for r in range(min_combinations, max_combinations + 1):
        # Generate combinations for the remaining props, excluding required ones
        for combination in itertools.combinations(
            [idx for idx in df.index if idx not in required_indices], r
        ):
            # Combine required props with the generated combination
            full_combination = required_indices + list(combination)

            # Initialize values for this combination
            prob_product = 1
            house_prob_product = 1
            odds_product = 1
            ev_sum = 0

            # Calculate values for the full combination
            for idx in full_combination:
                prop_data = df.loc[idx]
                prob_product *= prop_data['PROB']
                odds_product *= analyze.american_to_decimal(prop_data['ODDS'])
                house_prob_product *= prop_data['HOUSE_PROB']
                ev_sum += prop_data['EV']
            
            potential_winnings = 1 * (odds_product - 1)
            
            # Only include parlays with a combined EV greater than min_props
            if ev_sum > min_props:
                parlays.append({
                    'COMBO': full_combination,
                    'COMBINED_PROB': prob_product,
                    'COMBINED_HOUSE_PROB': house_prob_product,
                    'COMBINED_EV': ev_sum,
                    'TO_WIN': potential_winnings,
                })
    
    # Raise an error if no parlays were generated
    if not parlays:
        raise RuntimeError("parlays is empty")
    
    return pd.DataFrame(parlays)

# def select_optimal_parlays(prop_df, max_permeation_rate, min_props, max_props):
#     parlays_df = generate_parlays(prop_df, min_props, max_props)
#     parlays_df = parlays_df.sort_values(by="COMBINED_EV", ascending=False).reset_index(drop=True)
#     parlays_df['PARLAY_ID'] = parlays_df.index
#     num_parlays = int(len(prop_df) / max_permeation_rate)
#     parlays_selected = []
#     prop_counts = {prop: 0 for prop in prop_df.index}

#     for _, parlay in parlays_df.iterrows():
#         can_add_parlay = True
#         seen = {}
#         for idx in parlay['COMBO']:
#             prop_row = prop_df.iloc[idx]
#             player = prop_row['PLAYER']
#             stat = prop_row['STAT']

#             if stat == 'points':
#                 if player in seen and seen[player] == 'fgm':
#                     print('A')
#                     can_add_parlay = False
#                     break
#             if stat == 'fgm':
#                 if player in seen and seen[player] == 'points':
#                     print('B')
#                     can_add_parlay = False
#                     break
#             if prop_counts[idx] >= num_parlays * max_permeation_rate:
#                     can_add_parlay = False
#                     break
#             seen[player] = stat


#         if can_add_parlay:
#             parlays_selected.append(parlay)
#             for idx in parlay['COMBO']:
#                 prop_counts[idx] += 1
#     return parlays_selected


def calculate_num_parlays(prop_df, max_permeation_rate, min_props, max_props):
    """
    Calculate the optimal number of parlays based on given constraints.
    
    Args:
        prop_df (pd.DataFrame): DataFrame containing prop information.
        max_permeation_rate (float): Maximum rate at which a prop can be permeated.
        min_props (int): Minimum number of props in a parlay.
        max_props (int): Maximum number of props in a parlay.
        
    Returns:
        int: Optimal number of parlays.
    """
    
    # Calculate the average parlay size
    avg_parlay_size = (min_props + max_props) / 2
    
    # Adjust the calculation to consider the average parlay size
    num_parlays = int(len(prop_df) / (avg_parlay_size * max_permeation_rate))
    
    return num_parlays

def select_optimal_parlays(prop_df, max_permeation_rate, player_permeation_rate, min_props, max_props, force_props):
    """
    Selects optimal parlays based on given constraints.
    
    Args:
        prop_df (pd.DataFrame): DataFrame containing prop information with columns 'PLAYER', 'STAT', etc.
        max_permeation_rate (float): Maximum rate at which a prop can be permeated.
        player_permeation_rate (float): Maximum rate at which a player can be permeated.
        min_props (int): Minimum number of props in a parlay.
        max_props (int): Maximum number of props in a parlay.
        force_props (list): List of props that should not be limited by max_permeation_rate or player_permeation_rate.
    
    Returns:
        list: List of selected parlays.
    """
    
    # Generate all possible parlays
    parlays_df = generate_parlays(prop_df, min_props, max_props, force_props)

    # Sort parlays by combined expected value
    parlays_df = parlays_df.sort_values(by="COMBINED_EV", ascending=False).reset_index(drop=True)
    # Assign unique ID to each parlay
    parlays_df['PARLAY_ID'] = parlays_df.index
    
    # Calculate the maximum number of parlays using the updated calculation
    num_parlays = calculate_num_parlays(prop_df, max_permeation_rate, min_props, max_props)
    
    # Initialize counts
    parlays_selected = []
    prop_counts = {prop: 0 for prop in prop_df.index}
    player_counts = {player: 0 for player in prop_df['PLAYER']}
    
    # Select parlays
    for _, parlay in parlays_df.iterrows():
        can_add_parlay = True
        seen = {}
        
        for idx in parlay['COMBO']:
            prop_row = prop_df.iloc[idx]
            player = prop_row['PLAYER']
            stat = prop_row['STAT']

            # Check for conflicts in stat types
            if stat == 'points':
                if player in seen and 'fgm' in seen[player]:
                    can_add_parlay = False
                    break
            elif stat == 'fgm':
                if player in seen and 'points' in seen[player]:
                    can_add_parlay = False
                    break

            # Skip permeation checks if the prop or its player is in force_props
            if idx not in force_props:
                # Check for prop permeation limit
                if prop_counts[idx] >= num_parlays * max_permeation_rate:
                    can_add_parlay = False
                    break

                # Check for player permeation limit
                if player_counts[player] >= num_parlays * player_permeation_rate:
                    can_add_parlay = False
                    break

            # Track stats seen for the player in this parlay
            if player in seen:
                seen[player] += [stat]
            else:
                seen[player] = [stat]
        
        if can_add_parlay:
            parlays_selected.append(parlay)
            for idx in parlay['COMBO']:
                # Only update counts for non-force props and players
                if idx not in force_props:
                    prop_counts[idx] += 1
                    player_counts[prop_df.loc[idx, 'PLAYER']] += 1

    return parlays_selected


# def get_selected_parlays(n_props, max_prop_permeation_rate, max_player_permeation_rate, min_props_in_parlay, max_props_in_parlay, filter_players):
#     available_props = process_all_csv_files_in_directory(r"E:\coding_projects\nba_01\prop_lines\11032024")
#     analyzed_props = get_analyzed_props(available_props)
#     selected_props = dm.filter_props(analyzed_props, filter_players, n_props).reset_index(drop=True)
#     selected_parlays = select_optimal_parlays(selected_props, max_prop_permeation_rate, max_player_permeation_rate, min_props_in_parlay, max_props_in_parlay)
#     selected_parlay_dfs = []
#     for parlay in selected_parlays:
#         parlay_rows = []
#         for i in parlay['COMBO']:
#             prop = selected_props.iloc[i]
#             parlay_rows.append(prop)
#         parlay_df = pd.concat(parlay_rows, axis=1, ignore_index=True).T
#         parlay_df['PARLAY_EV'] = parlay['COMBINED_EV']
#         parlay_df['PARLAY_PROB'] = parlay['COMBINED_PROB']
#         parlay_df['PARLAY_HOUSE_PROB'] = parlay['COMBINED_HOUSE_PROB']
#         parlay_df['PARLAY_EV'] = parlay['COMBINED_EV']
#         parlay_df['PARLAY_ID'] = parlay['PARLAY_ID']
#         parlay_df['TO_WIN'] = parlay['TO_WIN']
#         selected_parlay_dfs.append(parlay_df)
#     selected_parlay_df = pd.concat(selected_parlay_dfs, axis=0, ignore_index=True)
#     selected_parlay_df['PROP_TAG'] = selected_parlay_df['PLAYER'] + "_" + selected_parlay_df['STAT'] + "_" + selected_parlay_df['TYPE']
#     return selected_parlay_df


def get_selected_parlays(n_props, max_prop_permeation_rate, max_player_permeation_rate, min_props_in_parlay, max_props_in_parlay, filter, force_props=None):
    available_props = process_all_csv_files_in_directory(r"E:\coding_projects\nba_01\prop_lines\11032024")
    analyzed_props = get_analyzed_props(available_props)
    selected_props = dm.filter_props(analyzed_props, filter, n_props, force_props).reset_index(drop=True)
    selected_parlays = select_optimal_parlays(selected_props, max_prop_permeation_rate, max_player_permeation_rate, min_props_in_parlay, max_props_in_parlay, force_props=force_props)

    selected_parlay_dfs = []
    for parlay in selected_parlays:
        parlay_rows = []
        for i in parlay['COMBO']:
            prop = selected_props.iloc[i]
            parlay_rows.append(prop)
        parlay_df = pd.concat(parlay_rows, axis=1, ignore_index=True).T
        parlay_df['PARLAY_EV'] = parlay['COMBINED_EV']
        parlay_df['PARLAY_PROB'] = parlay['COMBINED_PROB']
        parlay_df['PARLAY_HOUSE_PROB'] = parlay['COMBINED_HOUSE_PROB']
        parlay_df['PARLAY_EV'] = parlay['COMBINED_EV']
        parlay_df['PARLAY_ID'] = parlay['PARLAY_ID']
        parlay_df['TO_WIN'] = parlay['TO_WIN']
        selected_parlay_dfs.append(parlay_df)
    selected_parlay_df = pd.concat(selected_parlay_dfs, axis=0, ignore_index=True)
    selected_parlay_df['PROP_TAG'] = selected_parlay_df['PLAYER'] + "_" + selected_parlay_df['STAT'] + "_" + selected_parlay_df['TYPE']
    return selected_parlay_df


def get_prop_distribution(parlays):
    appearance_counts = parlays['PROP_TAG'].value_counts()
    parlay_count = len(parlays['PARLAY_ID'].unique())
    percentages = pd.Series(appearance_counts/parlay_count)
    parlay_distribution = pd.concat([appearance_counts, percentages], axis=1)
    parlay_distribution.columns = ['COUNT', '%']
    return parlay_distribution


def get_player_distribution(parlays):
    appearance_counts = parlays.groupby('PLAYER')['PARLAY_ID'].nunique().reset_index()
    appearance_counts.columns = ['PLAYER', 'PARLAY_COUNT']
    parlay_count = len(parlays['PARLAY_ID'].unique())
    percentages = pd.Series(appearance_counts['PARLAY_COUNT']/parlay_count)
    parlay_distribution = pd.concat([appearance_counts, percentages], axis=1)
    parlay_distribution.columns = ['PLAYER', 'PARLAY_COUNTS', '%']
    parlay_distribution = parlay_distribution.sort_values(by="PARLAY_COUNTS", ascending=False)
    return parlay_distribution


def create_pivot_table_for_tracking(df):
    pivot_df = df.assign(value=1).pivot_table(index=['PROP_TAG', 'THRESH'], columns='PARLAY_ID', values='value', fill_value="")

    # Reset index to move PROP_TAG and THRESH to columns
    pivot_df = pivot_df.reset_index()
    return pivot_df



In [2]:
filters_list = [

]

filter = pd.DataFrame(filters_list, columns=["name", "stat", "bet_type"])


In [3]:
force_props = [


]
force_props = pd.DataFrame(force_props, columns=["name", "stat", "bet_type"])

In [4]:
n_props = 60
max_prop_permeation_rate = 0.3
max_player_permeation_rate = 0.3
min_props_in_parlay = 4
max_props_in_parlay = 7

selected_parlays = get_selected_parlays(n_props, max_prop_permeation_rate, max_player_permeation_rate, min_props_in_parlay, max_props_in_parlay, filter, force_props=force_props)
prop_distribution = get_prop_distribution(selected_parlays)
player_distribution = get_player_distribution(selected_parlays)



E:\coding_projects\nba_01\prop_lines\11032024\ATL NO.csv
E:\coding_projects\nba_01\prop_lines\11032024\DET IND.csv
E:\coding_projects\nba_01\prop_lines\11032024\ORL DAL.csv
Matched instances: []


KeyboardInterrupt: 

In [ ]:
print(player_distribution)
print(prop_distribution)
print(len(selected_parlays['PARLAY_ID'].unique()))


                PLAYER  PARLAY_COUNTS         %
0        Anthony Black              9  1.000000
4          Jalen Suggs              9  1.000000
6        Jose Alvarado              9  1.000000
8     Tim Hardaway Jr.              9  1.000000
10  Wendell Carter Jr.              9  1.000000
9        Tobias Harris              8  0.888889
7         Kyrie Irving              3  0.333333
2           Jaden Ivey              2  0.222222
5       Jordan Hawkins              2  0.222222
1       Brandon Ingram              1  0.111111
3        Jalen Johnson              1  0.111111
                                 COUNT         %
PROP_TAG                                        
Wendell Carter Jr._points_over       9  1.000000
Jalen Suggs_points_over              9  1.000000
Jose Alvarado_points_over            9  1.000000
Tim Hardaway Jr._rebounds_under      9  1.000000
Anthony Black_assists_over           9  1.000000
Tobias Harris_rebounds_over          8  0.888889
Kyrie Irving_fg3m_over          

In [ ]:
today_dir = dt_mng.get_or_create_directory_in_days()
folder_name = dm.create_directory(f"{today_dir}/parlays/{datetime.now().strftime('%Y%m%d%H%M%S')}_study")
selected_parlays.to_csv(f'{folder_name}/parlays.csv')

Directory already exists: e:\coding_projects\nba_01\days
Directory already exists: e:\coding_projects\nba_01\days\2024-11-03
Directory 'e:\coding_projects\nba_01\days\2024-11-03/parlays/20241103132033_study' created successfully.


In [ ]:
print(selected_parlays.columns)

Index(['PLAYER', 'TEAM', 'STAT', 'THRESH', 'ODDS', 'TYPE', 'PROB', 'EV',
       'HOUSE_PROB', 'PARLAY_EV', 'PARLAY_PROB', 'PARLAY_HOUSE_PROB',
       'PARLAY_ID', 'TO_WIN', 'PROP_TAG'],
      dtype='object')


In [ ]:
players = selected_parlays['PLAYER'].unique()
game_data_players = []
for player in players:
    player_id = dm.get_player_id(player)
    player_game_data = dm.get_and_save_player_data(player_id)
    game_data_players.append(player_game_data)

teams = selected_parlays['TEAM'].unique()
game_data_teams = []
for team in teams:
    team_id = dm.get_team_id(team)
    team_game_data = dm.get_and_save_team_data(team_id)
    game_data_teams.append(team_game_data)

props = selected_parlays.drop_duplicates()[['PROP_TAG', 'THRESH', 'PARLAY_ID', 'PARLAY_EV', 'PARLAY_PROB', 'PARLAY_HOUSE_PROB', 'PROB', 'HOUSE_PROB', 'EV', 'TO_WIN']]
game_data = dict(zip(players, game_data_players))
for i, team in enumerate(teams):
    game_data[team] = game_data_teams[i]
dm.save_as_excel_workbook(game_data, folder_name + "/study_packet")
props.to_csv(folder_name + "/props.csv")

Wendell Carter Jr.             player_name player_position  minutes  points  rebounds  assists  \
325  Wendell Carter Jr.             C-F     28.0      14         7        2   
324  Wendell Carter Jr.             C-F     31.0      11        10        1   
326  Wendell Carter Jr.             C-F     30.0       3        10        2   
323  Wendell Carter Jr.             C-F     31.0       8        11        4   
322  Wendell Carter Jr.             C-F     26.0       8        14        2   
..                  ...             ...      ...     ...       ...      ...   
95   Wendell Carter Jr.             C-F     17.0       7         4        1   
96   Wendell Carter Jr.             C-F     29.0       6         7        2   
99   Wendell Carter Jr.             C-F     31.0       4         9        4   
89   Wendell Carter Jr.             C-F     17.0       8         2        1   
77   Wendell Carter Jr.             C-F     19.0       8         3        3   

       efg  fg3a  fg3m  fg3_pct 

In [ ]:
pivot_df = create_pivot_table_for_tracking(selected_parlays)
pivot_df.to_csv(f"{folder_name}/selected_parlays_as_pivot.csv")